In [ ]:
import pandas as pd
import pickle
import re
from  sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from datetime import *
from statistics import mean
from random import seed
from random import randint

In [ ]:
user_decision = int(input('Enter 1 to read Chicago dataset and enter 2 to read Las-Vegas dataset'))

In [ ]:
if(user_decision==1):
    df = pd.read_csv('chicago_data_frame_with_fewer_rows.csv')
    df = df.set_index(['hotel_id', 'reviewer_id'])
    with open("reviews_file_chicago.txt", "rb") as fp:   # Unpickling
        review_count = pickle.load(fp)
elif(user_decision ==2):
    df = pd.read_csv('las-vegas_data_frame_with_fewer_rows.csv')
    df = df.set_index(['hotel_id', 'reviewer_id'])
    with open("reviews_file_las-vegas.txt", "rb") as fp:   # Unpickling
        review_count = pickle.load(fp)

In [ ]:
regex = r'<hotelUrl>\n(.*?)\n</hotelUrl>'
unique_hotel_ids = set()
for i in review_count:
    hotel_name = re.search(regex, i)
    hotel_name = hotel_name[1]
    unique_hotel_ids.add(hotel_name)

In [ ]:
def convert_to_date(date):
    ans = datetime.strptime(date, "%b %d, %Y")
    return ans

In [ ]:
regex = r'<hotelUrl>\n(.*?)\n</hotelUrl>\n[\s\S]+?\n<helpfulness>\n(.*?)\n</helpfulness>'
unique_hotel_ids = set()
for i in review_count:
    hotel_name = re.search(regex, i)
    hotel_id = hotel_name[1]
    helpful_score = (hotel_name[2]).split(' of ')
    if (int(helpful_score[1])) > 4:
        unique_hotel_ids.add(hotel_id)


In [ ]:
date_member_hotel = []
obj = []
count = 0
date_member_hotel_regex = r'<memberUrl>\n([\s\S]*?)\n</memberUrl>\n<hotelUrl>\n([\s\S]*?)\n</hotelUrl>\n[\s\S]*?\n<date>\n(.*?)\n</date>'
for i in review_count:
    matched = re.search(date_member_hotel_regex, i)
    member = matched[1]
    hotel = matched[2]
    date = matched[3]
    obj.append([hotel, member, date])

In [ ]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [ ]:
def removekey(d, key):
    r = dict(d)
    r.pop(key)
    return r

In [ ]:
rows_with_hotel_as_key_culled_neg = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        neg = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) < 4:
                neg+=1
        if neg > 4:
            rows_with_hotel_as_key_culled_neg[i] = rows_with_hotel_as_key[i]

In [ ]:
len(rows_with_hotel_as_key_culled_neg)

In [ ]:
rows_with_hotel_as_key_culled_pos = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        pos = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) >= 4:
                pos+=1
        if pos > 4:
            rows_with_hotel_as_key_culled_pos[i] = rows_with_hotel_as_key[i]

In [ ]:
len(rows_with_hotel_as_key_culled_pos)

In [ ]:
x_train = pd.DataFrame(columns=df.columns)
x_train = pd.DataFrame(columns=df.columns)
x_train['hotel_id'] = ''
x_train['reviewer_id'] = ''
x_train = x_train.set_index(['hotel_id', 'reviewer_id'])

In [ ]:
classifier = RandomForestClassifier()
hotel_prediction = dict()
y_pred = []
y_test = []
y_pred_df_neg = dict()
for i in rows_with_hotel_as_key_culled_neg:
    #create a dataframe I can use to train using leave one out approach
    # i is the hotel we want to use to test the classifier which is trained using the other hotels
    x_test = rows_with_hotel_as_key_culled_neg[i].copy()
    y_train = []
    x_train = pd.DataFrame(columns=df.columns)
    for j in rows_with_hotel_as_key_culled_neg:
        if i != j:
            new_df = rows_with_hotel_as_key_culled_neg[j].copy()
            x_train = pd.concat([x_train, new_df])
    y_train = x_train.pop('helpful_of_not')
    y_train = (y_train.to_frame()['helpful_of_not']).astype(str).astype(int)
    classifier.fit(X=x_train.values, y=y_train.values)
    y_test.append(x_test.pop('helpful_of_not'))
    prediction = classifier.predict_proba(x_test)
    y_pred_df_neg[x_test.index[0][0]] = pd.DataFrame(prediction, columns = ['not helpful', 'helpfulness'], index=x_test.index)


In [ ]:
for i in y_pred_df_neg:
    y_pred_df_neg[i] = pd.merge(y_pred_df_neg[i], df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [ ]:
y_pred_fin_neg = dict()

for i in (y_pred_df_neg):
    count =0
    for j,row in y_pred_df_neg[i].iterrows():
        y_pred_fin_neg[i] = y_pred_df_neg[i][y_pred_df_neg[i]['ST1'] < 4]

In [ ]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['helpfulness'], inplace=True, ascending=False)

In [ ]:
neg_class_avg = []

for i in (y_pred_fin_neg):
    top = y_pred_fin_neg[i].iloc[0]
    neg_class_avg.append((top['helpfulness']))

In [ ]:
neg_class_total = []

for i in (y_pred_fin_neg):
    top = y_pred_fin_neg[i].iloc[0]
    helpful = (top['helpfulness'])
    if helpful >= 0.5:
        neg_class_total.append(1)
    else:
        neg_class_total.append(0)

In [ ]:
mean(neg_class_avg)

In [ ]:
mean(neg_class_total)

In [ ]:
x_train_pos = pd.DataFrame(columns=df.columns)
x_train_pos = pd.DataFrame(columns=df.columns)
x_train_pos['hotel_id'] = ''
x_train_pos['reviewer_id'] = ''
x_train_pos = x_train_pos.set_index(['hotel_id', 'reviewer_id'])

In [ ]:
classifier = RandomForestClassifier(n_estimators=39, max_depth=20)
hotel_prediction = dict()
y_pred = []
y_test = []
y_pred_df_pos = dict()
for i in rows_with_hotel_as_key_culled_pos:
    #create a dataframe I can use to train using leave one out approach
    # i is the hotel we want to use to test the classifier which is trained using the other hotels
    x_test = rows_with_hotel_as_key_culled_pos[i].copy()
    y_train = []
    x_train = pd.DataFrame(columns=df.columns)
    for j in rows_with_hotel_as_key_culled_pos:
        if i != j:
            new_df = rows_with_hotel_as_key_culled_pos[j].copy()
            x_train_pos = pd.concat([x_train, new_df])
    y_train_pos = x_train_pos.pop('helpful_of_not')
    y_train = (y_train_pos.to_frame()['helpful_of_not']).astype(str).astype(int)
    classifier.fit(X=x_train_pos.values, y=y_train.values)
    y_test.append(x_test.pop('helpful_of_not'))
    prediction = classifier.predict_proba(x_test)
    y_pred_df_pos[x_test.index[0][0]] = pd.DataFrame(prediction, columns = ['not helpful', 'helpfulness'], index=x_test.index)
    

In [ ]:
for i in y_pred_df_pos:
    y_pred_df_pos[i] = pd.merge(y_pred_df_pos[i], df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [ ]:
y_pred_fin_pos = dict()

for i in (y_pred_df_pos):
    count =0
    for j,row in y_pred_df_pos[i].iterrows():
        y_pred_fin_pos[i] = y_pred_df_pos[i][y_pred_df_pos[i]['ST1'] >= 4]

In [ ]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['helpfulness'], inplace=True, ascending=False)

In [ ]:
pos_class_avg = []

for i in (y_pred_fin_pos):
    top = y_pred_fin_pos[i].iloc[0]
    pos_class_avg.append((top['helpfulness']))

In [ ]:
pos_class_total = []

for i in (y_pred_fin_pos):
    top = y_pred_fin_pos[i].iloc[0]
    helpful = (top['helpfulness'])
    if helpful >= 0.5:
        pos_class_total.append(1)
    else:
        pos_class_total.append(0)

In [ ]:
mean(pos_class_avg)

In [ ]:
mean(pos_class_total)

# Date Negative Mean Review Helpfulness

In [ ]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i]['date'] = ''

In [ ]:
for j in y_pred_fin_neg:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_neg[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_neg[j].set_value(y_pred_fin_neg[j].loc[[test_tuple]].index,'date',  date)

In [ ]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

In [ ]:
hold = y_pred_fin_neg.copy()
for i in hold:
    if len(hold[i]) < 1:
        del y_pred_fin_neg[i]

In [ ]:
values_helpful_of_not_neg = df['helpful_of_not']

In [ ]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i] = pd.merge(y_pred_fin_neg[i], values_helpful_of_not_neg, how = 'left', on=['hotel_id', 'reviewer_id'])

In [ ]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i] = pd.merge(y_pred_fin_neg[i], values_helpful_of_not_neg, how = 'left', on=['hotel_id', 'reviewer_id'])

In [ ]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

In [ ]:
total = []
for i in (y_pred_fin_neg):
    top = (y_pred_fin_neg[i].iloc[0])['helpful_of_not']
    total.append(top)

total_fin = list(map(int, total))

In [ ]:
mean(total_fin)

# Positive Date Mean Review Helpfulness

In [ ]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i]['date'] = ''

In [ ]:
for j in y_pred_fin_pos:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_pos[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_pos[j].set_value(y_pred_fin_pos[j].loc[[test_tuple]].index,'date',  date)

In [ ]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

In [ ]:
hold = y_pred_fin_pos.copy()
for i in hold:
    if len(hold[i]) < 1:
        del y_pred_fin_pos[i]

In [ ]:
values_helpful_of_not_pos = df['helpful_of_not']

In [ ]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i] = pd.merge(y_pred_fin_pos[i], values_helpful_of_not_pos, how = 'left', on=['hotel_id', 'reviewer_id'])

In [ ]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i] = pd.merge(y_pred_fin_pos[i], values_helpful_of_not_pos, how = 'left', on=['hotel_id', 'reviewer_id'])

In [ ]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

In [ ]:
total = []
for i in (y_pred_fin_pos):
    top = (y_pred_fin_pos[i].iloc[0])['helpful_of_not']
    total.append(top)

total_fin = list(map(int, total))

In [ ]:
mean(total_fin)

# Random Negative Percentage of recommended reviews that are helpful

In [ ]:
for i in y_pred_fin_neg:
    del y_pred_fin_neg[i]['date']

In [ ]:
for i in y_pred_fin_neg:
    rand_int = randint(0,len(y_pred_fin_neg[i])-1)
    top = (y_pred_fin_neg[i].iloc[rand_int])['helpful_of_not']
    total.append(top)

In [ ]:
total_fin_neg = list(map(int, total))

In [ ]:
mean(total_fin_neg)

In [ ]:
df.pop('helpful_of_not')

In [ ]:
if user_decision==1:
    vals_df = pd.read_csv('vals_df.csv')
    vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])
    df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])
else:
    vals_df = pd.read_csv('las-vegas_vals_df.csv')
    vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])
    df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [ ]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [ ]:
rows_with_hotel_as_key_culled_neg = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        neg = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) < 4:
                neg+=1
        if neg > 4:
            rows_with_hotel_as_key_culled_neg[i] = rows_with_hotel_as_key[i]

In [ ]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i]['date'] = ''

In [ ]:
for j in y_pred_fin_neg:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_neg[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_neg[j].set_value(y_pred_fin_neg[j].loc[[test_tuple]].index,'date',  date)

In [ ]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

In [ ]:
total = []
for i in (y_pred_fin_neg):
    top = (y_pred_fin_neg[i].iloc[0])['helpfulness']
    total.append(top)

In [ ]:
print(type(total))
total_fin = list(map(float, total))

In [ ]:
mean(total_fin)

# Random Positive Percentage of recommended reviews that are helpful

In [ ]:
for i in y_pred_fin_pos:
    rand_int = randint(0,len(y_pred_fin_pos[i])-1)
    top = (y_pred_fin_pos[i].iloc[rand_int])['helpful_of_not']
    total.append(top)

In [ ]:
total_fin_pos = list(map(int, total))

In [ ]:
mean(total_fin_pos)

In [ ]:
df.pop('helpful_of_not')

In [ ]:
if user_decision==1:
    vals_df = pd.read_csv('vals_df.csv')
    vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])
    df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])
else:
    vals_df = pd.read_csv('las-vegas_vals_df.csv')
    vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])
    df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [ ]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [ ]:
rows_with_hotel_as_key_culled_pos = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        pos = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) >= 4:
                pos+=1
        if pos > 4:
            rows_with_hotel_as_key_culled_pos[i] = rows_with_hotel_as_key[i]

In [ ]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i]['date'] = ''

In [ ]:
for j in y_pred_fin_pos:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_pos[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_pos[j].set_value(y_pred_fin_pos[j].loc[[test_tuple]].index,'date',  date)

In [ ]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

In [ ]:
total = []
for i in (y_pred_fin_pos):
    top = (y_pred_fin_pos[i].iloc[0])['helpfulness']
    total.append(top)

In [ ]:
print(type(total))
total_fin = list(map(float, total))

In [ ]:
mean(total_fin)